# Fake News Detection on Twitter - Inference

## Grab Tweet

In [66]:
tweet_id = input("Enter tweet ID: ") 

Enter tweet ID:  552783745565347840


In [32]:
consumer_key = ""
consumer_secret = ""
access_token = ""
access_token_secret = ""

In [33]:
! pip install tweepy

scikit-umfpack 0.3.2 has requirement numpy>=1.15.3, but you'll have numpy 1.15.1 which is incompatible.
menpo 0.8.1 has requirement matplotlib<2.0,>=1.4, but you'll have matplotlib 2.2.3 which is incompatible.
menpo 0.8.1 has requirement pillow<5.0,>=3.0, but you'll have pillow 5.2.0 which is incompatible.
menpo 0.8.1 has requirement scipy<1.0,>=0.16, but you'll have scipy 1.1.0 which is incompatible.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [34]:
import tweepy

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [67]:
twitter_api = tweepy.API(auth)
tweet = twitter_api.get_status(tweet_id)

In [55]:
def parse_tweet(raw_tweet):
    parsed_tweet = {}

    parsed_tweet["id"] = raw_tweet["id"]

    # Style
    parsed_tweet["text"] = raw_tweet["text"]
    parsed_tweet["num_mentions"] = len(raw_tweet["entities"]["user_mentions"])
    parsed_tweet["num_hashtags"] = len(raw_tweet["entities"]["hashtags"])
    parsed_tweet["num_urls"] = len(raw_tweet["entities"]["urls"])

    if "media" in raw_tweet["entities"].keys():
        parsed_tweet["has_media"] = True
    else:
        parsed_tweet["has_media"] = False

    # Propagation
    parsed_tweet["num_likes"] = raw_tweet["favorite_count"]
    parsed_tweet["num_retweets"] = raw_tweet["retweet_count"]

    # Credibility
    parsed_tweet["user_verified"] = raw_tweet["user"]["verified"]
    parsed_tweet["user_no_profile_image"] = raw_tweet["user"]["default_profile_image"]
    parsed_tweet["user_num_friends"] = raw_tweet["user"]["friends_count"]
    parsed_tweet["user_num_followers"] = raw_tweet["user"]["followers_count"]
    parsed_tweet["user_num_lists"] = raw_tweet["user"]["listed_count"]
    parsed_tweet["user_num_tweets"] = raw_tweet["user"]["statuses_count"]
    parsed_tweet["user_num_friends"] = raw_tweet["user"]["friends_count"]
    parsed_tweet["user_num_favourite_tweets"] = raw_tweet["user"]["favourites_count"]
    parsed_tweet["user_protected"] = raw_tweet["user"]["protected"]

    if raw_tweet["coordinates"] != None:
        parsed_tweet["has_location"] = True
    else:
        parsed_tweet["has_location"] = False

    # For filtering
    parsed_tweet["language"] = raw_tweet["lang"]

    parsed_tweet["annotation"] = None
    return parsed_tweet

In [68]:
parsed_tweet = parse_tweet(tweet._json)

In [69]:
import pandas as pd

tweets_df = pd.DataFrame.from_dict([parsed_tweet])

In [70]:
tweets_df

,annotation,has_location,has_media,id,language,num_hashtags,num_likes,num_mentions,num_retweets,num_urls,text,user_no_profile_image,user_num_favourite_tweets,user_num_followers,user_num_friends,user_num_lists,user_num_tweets,user_protected,user_verified
0,None,False,False,552783745565347840,en,1,12,0,99,0,Ten killed in shooting at headquarters of Fren...,False,520,2277592,4525,10286,100783,False,True


## Text Model

In [59]:
from pathlib import Path

path = Path('./')

In [60]:
from fastai.text import *

In [61]:
learn = load_learner(path, file = 'model_clas.pkl')

In [71]:
print(learn.predict(tweet.text))

(Category True, tensor(1), tensor([0.0858, 0.9142]))


## Tabular Model

In [63]:
from fastai.tabular import *

In [64]:
learn_tab = load_learner(path, file = 'model_clas_tab.pkl')

In [72]:
print(learn_tab.predict(tweets_df.iloc[0]))

(Category True, tensor(1), tensor([0.1697, 0.8303]))
